In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib widget

In [ ]:
from lsst.utils.plotting import publication_plots
from lsst.utils.plotting import get_multiband_plot_colors

colors = get_multiband_plot_colors()
bands = colors.keys()

In [ ]:
from lsst.summit.utils import (
    getAirmassSeeingCorrection,
    getBandpassSeeingCorrection,
)

In [ ]:
CAM_FWHM = 0.207 # arcsec

In [ ]:
#import os
#os.environ["no_proxy"] += ",.consdb"

from lsst.summit.utils import ConsDbClient

client = ConsDbClient("http://consdb-pq.consdb:8080/consdb")

In [ ]:
instrument = 'lsstcam'

In [ ]:
def fetch(day_obs_min, day_obs_max):
    
    visits_query = f'''
    SELECT 
    ccdvisit1_quicklook.psf_sigma,
    ccdvisit1_quicklook.psf_ixx,
    ccdvisit1_quicklook.psf_ixy,
    ccdvisit1_quicklook.psf_iyy,
    ccdvisit1.detector,
    visit1.visit_id,
    visit1.seq_num,
    visit1.band,
    visit1.physical_filter,
    visit1.day_obs,
    visit1.target_name,
    visit1.airmass,
    visit1.altitude,
    visit1.azimuth,
    visit1.sky_rotation,
    visit1.exp_midpt_mjd,
    visit1.dimm_seeing,
    visit1.s_ra,
    visit1.s_dec,
    visit1_quicklook.psf_sigma_min,
    visit1_quicklook.psf_sigma_median,
    visit1_quicklook.aos_fwhm,
    visit1_quicklook.donut_blur_fwhm,
    visit1_quicklook.ringss_seeing,
    visit1_quicklook.seeing_zenith_500nm_min,
    visit1_quicklook.seeing_zenith_500nm_median,
    visit1_quicklook.physical_rotator_angle
    FROM
    cdb_{instrument}.ccdvisit1_quicklook AS ccdvisit1_quicklook,
    cdb_{instrument}.ccdvisit1 AS ccdvisit1,
    cdb_{instrument}.visit1_quicklook AS visit1_quicklook,
    cdb_{instrument}.visit1 AS visit1 
    WHERE 
    ccdvisit1.ccdvisit_id = ccdvisit1_quicklook.ccdvisit_id
    AND ccdvisit1.visit_id = visit1.visit_id 
    AND visit1.visit_id = visit1_quicklook.visit_id
    AND ccdvisit1.detector NOT IN (168, 188, 123, 27, 0, 20, 65, 161)
    AND visit1.airmass > 0
    AND visit1.day_obs >= {day_obs_min} AND visit1.day_obs <= {day_obs_max}
    -- AND visit1.science_program in ('BLOCK-365', 'BLOCK-407');
    AND visit1.science_program in ('BLOCK-365', 'BLOCK-407', 'BLOCK-408');
    '''
    
    ccdvisits = client.query(visits_query).to_pandas()

    pixel_scale = 0.2
    sig2fwhm = 2 * np.sqrt(2 * np.log(2))
    ccdvisits["psf_fwhm"] = ccdvisits["psf_sigma"] * sig2fwhm * pixel_scale
    ccdvisits["psf_fwhm"] = pd.to_numeric(ccdvisits["psf_fwhm"], errors="coerce")
    ccdvisits["donut_blur_fwhm"] = pd.to_numeric(ccdvisits["donut_blur_fwhm"], errors="coerce")
    ccdvisits["aos_fwhm"] = pd.to_numeric(ccdvisits["aos_fwhm"], errors="coerce")
    
    #airmass_seeing_correction = np.array([getAirmassSeeingCorrection(airmass) for airmass in ccdvisits["airmass"]])
    #bandpass_seeing_correction = np.array([getBandpassSeeingCorrection(band) for band in ccdvisits["physical_filter"]])
    #ccdvisits["psf_fwhm_zenith_500nm"] = ccdvisits["psf_fwhm"] * airmass_seeing_correction * bandpass_seeing_correction

    #ccdvisits["psf_e1"] = (ccdvisits["psf_ixx"]**2 - ccdvisits["psf_iyy"]**2) / (ccdvisits["psf_ixx"]**2 + ccdvisits["psf_iyy"]**2)
    #ccdvisits["psf_e2"] = (2. * ccdvisits["psf_ixy"]**2) / (ccdvisits["psf_ixx"]**2 + ccdvisits["psf_iyy"]**2)
    ccdvisits["psf_e1"] = (ccdvisits["psf_ixx"] - ccdvisits["psf_iyy"]) / (ccdvisits["psf_ixx"] + ccdvisits["psf_iyy"])
    ccdvisits["psf_e2"] = (2. * ccdvisits["psf_ixy"]) / (ccdvisits["psf_ixx"] + ccdvisits["psf_iyy"])
    ccdvisits["psf_e"] = np.sqrt(np.array(ccdvisits["psf_e1"]**2 + ccdvisits["psf_e2"]**2, dtype=float))

    return ccdvisits

In [ ]:
#day_obs_min = 20250620
#day_obs_max = 20250709
#day_obs_min = 20250906
#day_obs_max = 20250906
#day_obs_min = 20250708
#day_obs_max = 20250708
#day_obs_min = 20250709
#day_obs_max = 20250709
#day_obs_min = 20250913
#day_obs_max = 20250913
#day_obs_min = 20250915
#day_obs_max = 20250915
#day_obs_min = 20250921
#day_obs_max = 20250921
day_obs_min = 20251026
#day_obs_max = 20251026
#day_obs_min = 20251027
#day_obs_max = 20251027
#day_obs_min = 20251029
#day_obs_max = 20251029
#day_obs_max = 20251102
#day_obs_min = 20251104
#day_obs_min = 20251109
#day_obs_max = 20251201
#day_obs_min = 20251114
day_obs_max = 20251117
ccdvisits = fetch(day_obs_min, day_obs_max)

In [ ]:
airmass_seeing_correction = np.array([getAirmassSeeingCorrection(airmass) for airmass in ccdvisits["airmass"]])
bandpass_seeing_correction = np.array([getBandpassSeeingCorrection(band) for band in ccdvisits["physical_filter"]])
ccdvisits["psf_fwhm_zenith_500nm"] = ccdvisits["psf_fwhm"] * airmass_seeing_correction * bandpass_seeing_correction

In [ ]:
ccdvisits['donut_blur_atm_fwhm'] = np.sqrt(ccdvisits['donut_blur_fwhm']**2 - CAM_FWHM**2)
ccdvisits['aos_cam_fwhm'] = np.sqrt(ccdvisits['aos_fwhm']**2 + CAM_FWHM**2)

In [ ]:
ccdvisits

In [ ]:
selection = (ccdvisits['band'] != "y") & (ccdvisits['day_obs'] >= 20251026)
print(np.sum(selection))

plt.figure()
plt.hexbin(
    ccdvisits["donut_blur_atm_fwhm"][selection], 
    ccdvisits["psf_fwhm"][selection], 
    #ccdvisits["psf_fwhm_zenith_500nm"][selection], 
    gridsize=50, mincnt=10, extent=(0.5, 2.0, 0.5, 2.0), cmap='viridis')
plt.colorbar(label='Density')
plt.plot(np.linspace(0.5, 2.0, 10), np.linspace(0.5, 2.0, 10), lw=2, ls='--', c='black')

x = np.linspace(0.5, 2.0, 100)
plt.plot(x, np.sqrt(x**2 + 0.4**2), c='red', lw=2, ls='--', label='0.4" System Contribution')
plt.plot(x, np.sqrt(x**2 + 0.5**2), c='orange', lw=2, ls='--', label='0.5" System Contribution')
plt.plot(x, np.sqrt(x**2 + 0.6**2), c='cyan', lw=2, ls='--', label='0.6" System Contribution')
plt.plot(x, np.sqrt(x**2 + 0.7**2), c='magenta', lw=2, ls='--', label='0.7" System Contribution')

plt.xlim(0.5, 1.8)
plt.ylim(0.5, 1.8)

plt.legend(loc='lower right')

#plt.xlabel("$\sqrt{ {\rm Donut_Blur_FWHM}^2 - {\rm CAM_FWHM}^2}$ (arcsec)")
plt.xlabel("$\sqrt{\mathrm{DONUT\_BLUR\_FWHM}^2 - \mathrm{CAM\_FWHM}^2}$ (arcsec)")
#plt.ylabel("psf_fwhm_zenith_500nm")
plt.ylabel("PSF FWHM (arcsec)")
plt.title("Using Donut Blur as Proxy for Atmosphere Contribution")
#plt.show()

In [ ]:
bins = np.linspace(0., 2., 41)
plt.figure()
plt.hist(
    np.sqrt(ccdvisits["psf_fwhm"][selection]**2 + CAM_FWHM**2 - ccdvisits["donut_blur_fwhm"][selection]**2), 
    bins=bins, histtype='step', label='w/ CAM_FWHM',
)
plt.hist(
    np.sqrt(ccdvisits["psf_fwhm"][selection]**2 - ccdvisits["donut_blur_fwhm"][selection]**2), 
    bins=bins, histtype='step', label='w/out CAM_FWHM',
)
plt.legend()

In [ ]:
bins = np.linspace(0., 2., 41)
plt.figure()
plt.hist(
    ccdvisits["aos_fwhm"][selection], 
    bins=bins, histtype='step', label='w/out CAM_FWHM',
)
plt.hist(
    ccdvisits["aos_cam_fwhm"], 
    bins=bins, histtype='step', label='w/ CAM_FWHM',
)
plt.legend()

In [ ]:
"""
selection = (ccdvisits['seq_num'] == 52) 

bins = np.linspace(-0.5, 0.5, 150)
plt.figure()
plt.hist(ccdvisits['psf_e1'][selection], bins=bins, histtype='step', label='e1')
plt.hist(ccdvisits['psf_e2'][selection], bins=bins, histtype='step', label='e2')
plt.hist(ccdvisits['psf_e'][selection], bins=bins, histtype='step', label='e')
plt.legend()
"""

In [ ]:
bins = np.linspace(-0.4, 0.4, 81)

plt.figure()
plt.hist(ccdvisits['psf_e'], bins=bins, density=True, histtype='step', lw=2, label='PSF e')
plt.hist(ccdvisits['psf_e1'], bins=bins, density=True, histtype='step', lw=2, label='PSF e1')
plt.hist(ccdvisits['psf_e2'], bins=bins, density=True, histtype='step', lw=2, label='PSF e2')
plt.legend()
plt.xlabel('Ellipticity')
plt.ylabel('PDF')
plt.xlim(bins[0], bins[-1])
plt.ylim(0., plt.ylim()[-1])
plt.title('day_obs: %i - %s'%(min(ccdvisits['day_obs']), max(ccdvisits['day_obs'])))
plt.show()

In [ ]:
groups = ccdvisits.groupby('visit_id')
visits_summary = pd.DataFrame({
    'day_obs': groups['day_obs'].first(),
    'seq_num': groups['seq_num'].median(),
    'donut_blur_fwhm': groups['donut_blur_fwhm'].median(),
    'aos_fwhm': groups['aos_fwhm'].median(),
    'donut_blur_atm_fwhm': groups['donut_blur_atm_fwhm'].median(),
    'aos_cam_fwhm': groups['aos_cam_fwhm'].median(),
    'physical_rotator_angle': groups['physical_rotator_angle'].median(),
    'altitude': groups['altitude'].median(),
    'psf_fwhm_05': groups['psf_fwhm'].quantile(0.05),
    'psf_fwhm_50': groups['psf_fwhm'].quantile(0.50),
    'psf_fwhm_95': groups['psf_fwhm'].quantile(0.95),
    'psf_fwhm_zenith_500nm_50': groups['psf_fwhm_zenith_500nm'].quantile(0.50),
    'psf_e_05': groups['psf_e'].quantile(0.05),
    'psf_e_50': groups['psf_e'].quantile(0.50),
    'psf_e_95': groups['psf_e'].quantile(0.95),
    'band': groups['band'].first(),
})
visits_summary['psf_fwhm_95_05'] = np.sqrt(visits_summary['psf_fwhm_95']**2 - visits_summary['psf_fwhm_05']**2)
visits_summary['sys_50'] = np.sqrt(visits_summary['psf_fwhm_50']**2 + CAM_FWHM**2 - visits_summary['donut_blur_fwhm']**2)

In [ ]:
visits_summary['band']

In [ ]:
plt.figure()
plt.scatter(visits_summary['psf_fwhm_50'], visits_summary['psf_e_50'], s=2)
#plt.scatter(visits_summary['donut_blur_fwhm'], visits_summary['psf_e_50'], s=2)
#plt.scatter(visits_summary['psf_fwhm_95_05'], visits_summary['psf_e_50'], s=2)
plt.xlabel('Per-visit Median PSF FWHM (arcsec)')
plt.ylabel('Per-visit Mediain PSF e')

In [ ]:
visits_summary

In [ ]:
visits_summary['day_obs'].value_counts()

In [ ]:
np.mean(visits_summary['day_obs'].value_counts())

In [ ]:
plt.figure()
plt.scatter(visits_summary['aos_fwhm'], visits_summary['psf_fwhm_95_05'], s=1)
#plt.xlim(0., 1.)
#plt.ylim(0., 1.)

In [ ]:
plt.figure()
plt.hist(visits_summary['psf_fwhm_50'], bins=41)

In [ ]:
selection = (visits_summary['band'] != "y") & (visits_summary['day_obs'] >= 20251026)
print(np.sum(selection))

plt.figure()
plt.hexbin(
    visits_summary["donut_blur_atm_fwhm"][selection], 
    visits_summary["psf_fwhm_50"][selection], 
    #visits_summary["psf_fwhm_zenith_500nm_50"][selection], 
    gridsize=40, mincnt=1, extent=(0.5, 2.0, 0.5, 2.0), cmap='viridis')
plt.colorbar(label='Density')
plt.plot(np.linspace(0.5, 2.0, 10), np.linspace(0.5, 2.0, 10), lw=2, ls='--', c='black')

x = np.linspace(0.5, 2.0, 100)
plt.plot(x, np.sqrt(x**2 + 0.4**2), c='red', lw=2, ls='--', label='0.4" System Contribution')
plt.plot(x, np.sqrt(x**2 + 0.5**2), c='orange', lw=2, ls='--', label='0.5" System Contribution')
plt.plot(x, np.sqrt(x**2 + 0.6**2), c='cyan', lw=2, ls='--', label='0.6" System Contribution')
plt.plot(x, np.sqrt(x**2 + 0.7**2), c='magenta', lw=2, ls='--', label='0.7" System Contribution')

plt.xlim(0.5, 1.8)
plt.ylim(0.5, 1.8)

plt.xlabel("Donut Blur FWHM (arcsec)")
#plt.ylabel("psf_fwhm_zenith_500nm")
plt.ylabel("PSF FWHM (arcsec)")
plt.show()

## Individual Day Summary

In [ ]:
day_obs = 20251117
selection = (visits_summary['day_obs'] == day_obs)

print(np.sum(selection))

fig, ax = plt.subplots(3, sharex=True, figsize=(16, 10))

ax[0].errorbar(visits_summary['seq_num'][selection], 
               visits_summary['psf_fwhm_50'][selection],
               yerr=np.array([
                   visits_summary['psf_fwhm_50'][selection] - visits_summary['psf_fwhm_05'][selection], 
                   visits_summary['psf_fwhm_95'][selection] - visits_summary['psf_fwhm_50'][selection],
               ]), 
               fmt='_', label='PSF FWHM: 5, 50, 95 percentile')
ax[0].errorbar(visits_summary['seq_num'][selection], 
               visits_summary['psf_e_50'][selection],
               yerr=np.array([
                   visits_summary['psf_e_50'][selection] - visits_summary['psf_e_05'][selection], 
                   visits_summary['psf_e_95'][selection] - visits_summary['psf_e_50'][selection],
               ]), 
               fmt='_', label='PSF e: 5, 50, 95 percentile')
#ax[0].scatter(visits_summary['seq_num'], visits_summary['psf_fwhm_95'])
#ax[0].scatter(visits_summary['seq_num'], visits_summary['psf_fwhm_50'])
#ax[0].scatter(visits_summary['seq_num'], visits_summary['psf_fwhm_05'])
ax[0].scatter(visits_summary['seq_num'][selection], visits_summary['psf_fwhm_95_05'][selection], c='red', marker='x', label='sqrt(fwhm_95^2 - fwhm_5^2)')
ax[0].scatter(visits_summary['seq_num'][selection], visits_summary['donut_blur_fwhm'][selection], c='black', marker='o', label='Donut Blur')
ax[0].scatter(visits_summary['seq_num'][selection], visits_summary['aos_fwhm'][selection], c='green', marker='+', label='AOS Residual')
ax[0].set_ylabel('Delivered Image Quality Metric\n(arcsec)')
#ax[0].scatter(visit_seq_num, np.sqrt(visit_donut_blur_fwhm**2 + visit_psf_fwhm_95_05**2), c='green', marker='+', label='AOS Residual')
ax[0].legend()
ax[0].set_ylim(0., 2.)
ax[0].grid(True)

for band in bands:
    band_selection = selection & (visits_summary["band"] == band)
    ax[1].scatter(visits_summary['seq_num'][band_selection], visits_summary['altitude'][band_selection], marker='.', s=50, c=colors[band], label=band)
#ax[1].scatter(visits_summary['seq_num'], visits_summary['altitude'], marker='.', s=10, c='black')
ax[1].set_ylabel('Elevation\n(deg)')
ax[1].grid(True)
ax[1].legend()

for band in bands:
    band_selection = selection & (visits_summary["band"] == band)
    ax[2].scatter(visits_summary['seq_num'][band_selection], visits_summary['physical_rotator_angle'][band_selection], marker='.', s=50, c=colors[band], label=band)
#ax[2].scatter(visits_summary['seq_num'], visits_summary['physical_rotator_angle'], marker='.', s=10, c='black')
ax[2].set_ylabel('Phys Rot Angle\n(deg)')
ax[2].grid(True)

ax[-1].set_xlabel('seq_num')

plt.suptitle('day_obs = %s'%(day_obs))

#plt.suptitle('day_obs = %s'%(day_obs_min))
plt.subplots_adjust(hspace=0)
plt.tight_layout()

In [ ]:
selection = np.isin(visits_summary['day_obs'], [20251026])

bins = np.linspace(0., 1.5, 40)

plt.figure()
#plt.hist(visits_summary['aos_fwhm'], bins=bins, density=True, histtype='step')
#plt.hist(visits_summary['aos_fwhm'][selection], bins=bins, density=True, histtype='step')
plt.hist(visits_summary['psf_fwhm_95_05'], bins=bins, density=True, histtype='step', lw=2, label='All (nvisits = %i)'%(len(selection)))
plt.hist(visits_summary['psf_fwhm_95_05'][selection], bins=bins, density=True, histtype='step', lw=2, label='20251026 (nvisits = %i)'%(np.sum(selection)))
plt.xlabel('sqrt(fwhm_95^2 - fwhm_5^2) (arcsec)')
plt.xlim(0., 1.5)
plt.legend()

In [ ]:
plt.figure()
plt.scatter(visits_summary['aos_fwhm'], visits_summary['psf_fwhm_95_05'], s=1)
plt.xlim(0.2, 1.2)
plt.ylim(0.2, 1.2)

## Day Obs Summary

In [ ]:
#for key in visits_summary.keys():
#    visits_summary[key] = pd.to_numeric(visits_summary[key], errors="coerce")

selection = (visits_summary['band'] != "y")

groups = visits_summary[selection].groupby('day_obs')
day_obs_summary = pd.DataFrame({
    'day_obs': groups['day_obs'].first(),
    'n_visits': groups['day_obs'].count(),
    'psf_fwhm_95_05_low': groups['psf_fwhm_95_05'].quantile(0.10),
    'psf_fwhm_95_05_50': groups['psf_fwhm_95_05'].quantile(0.50),
    'psf_fwhm_95_05_high': groups['psf_fwhm_95_05'].quantile(0.90),
    'aos_fwhm_low': groups['aos_fwhm'].quantile(0.10),
    'aos_fwhm_50': groups['aos_fwhm'].quantile(0.50),
    'aos_fwhm_high': groups['aos_fwhm'].quantile(0.90),
    'aos_cam_fwhm_low': groups['aos_cam_fwhm'].quantile(0.10),
    'aos_cam_fwhm_50': groups['aos_cam_fwhm'].quantile(0.50),
    'aos_cam_fwhm_high': groups['aos_cam_fwhm'].quantile(0.90),
    'sys_50_low': groups['sys_50'].quantile(0.10),
    'sys_50_50': groups['sys_50'].quantile(0.50),
    'sys_50_high': groups['sys_50'].quantile(0.90),
    'psf_e_50_low': groups['psf_e_50'].quantile(0.10),
    'psf_e_50_50': groups['psf_e_50'].quantile(0.50),
    'psf_e_50_high': groups['psf_e_50'].quantile(0.90),
    'psf_fwhm_50_low': groups['psf_fwhm_50'].quantile(0.10),
    'psf_fwhm_50_50': groups['psf_fwhm_50'].quantile(0.50),
    'psf_fwhm_50_high': groups['psf_fwhm_50'].quantile(0.90),
    'donut_blur_atm_fwhm_low': groups['donut_blur_atm_fwhm'].quantile(0.10),
    'donut_blur_atm_fwhm_50': groups['donut_blur_atm_fwhm'].quantile(0.50),
    'donut_blur_atm_fwhm_high': groups['donut_blur_atm_fwhm'].quantile(0.90),
})

In [ ]:
day_obs_summary

In [ ]:
selection = np.tile(True, len(day_obs_summary))

xticks = np.arange(len(day_obs_summary[selection]))

plt.figure(figsize=(10, 6))
plt.errorbar(xticks - 0.15, day_obs_summary['psf_fwhm_95_05_50'][selection],
             yerr=np.array([day_obs_summary['psf_fwhm_95_05_50'][selection] - day_obs_summary['psf_fwhm_95_05_low'][selection], 
                            day_obs_summary['psf_fwhm_95_05_high'][selection] - day_obs_summary['psf_fwhm_95_05_50'][selection]]), 
             fmt='_', c='tab:blue', label='sqrt(fwhm_95^2 - fwhm_5^2): 10th, 50th, 90th Percentile')
#plt.errorbar(xticks + 0.2, day_obs_summary['aos_fwhm_50'][selection],
#             yerr=np.array([day_obs_summary['aos_fwhm_50'][selection] - day_obs_summary['aos_fwhm_low'][selection], 
#                            day_obs_summary['aos_fwhm_high'][selection] - day_obs_summary['aos_fwhm_50'][selection]]), 
#             fmt='_', c='magenta', label='aos_fwhm: 10th, 50th, 90th Percentile')
plt.errorbar(xticks + 0., day_obs_summary['aos_cam_fwhm_50'][selection],
             yerr=np.array([day_obs_summary['aos_cam_fwhm_50'][selection] - day_obs_summary['aos_cam_fwhm_low'][selection], 
                            day_obs_summary['aos_cam_fwhm_high'][selection] - day_obs_summary['aos_cam_fwhm_50'][selection]]), 
             fmt='_', c='tab:purple', label='sqrt(aos_fwhm^2 + cam_fwhm^2): 10th, 50th, 90th Percentile')
plt.errorbar(xticks + 0.15, day_obs_summary['sys_50_50'][selection],
             yerr=np.array([day_obs_summary['sys_50_50'][selection] - day_obs_summary['sys_50_low'][selection], 
                            day_obs_summary['sys_50_high'][selection] - day_obs_summary['sys_50_50'][selection]]), 
             fmt='_', c='tab:red', label='sqrt(fwhm_50^2 - (donut_blur^2 - cam_fwhm^2)): 10th, 50th, 90th Percentile')
plt.errorbar(xticks + 0., day_obs_summary['psf_e_50_50'][selection],
             yerr=np.array([day_obs_summary['psf_e_50_50'][selection] - day_obs_summary['psf_e_50_low'][selection], 
                            day_obs_summary['psf_e_50_high'][selection] - day_obs_summary['psf_e_50_50'][selection]]), 
             fmt='_', c='tab:orange', label='psf_e: 10th, 50th, 90th Percentile')


plt.errorbar(xticks - 0.075, day_obs_summary['psf_fwhm_50_50'][selection],
             yerr=np.array([day_obs_summary['psf_fwhm_50_50'][selection] - day_obs_summary['psf_fwhm_50_low'][selection], 
                            day_obs_summary['psf_fwhm_50_high'][selection] - day_obs_summary['psf_fwhm_50_50'][selection]]), 
             fmt='_', c='black', label='fwhm_50: 10th, 50th, 90th Percentile')
plt.errorbar(xticks + 0.075, day_obs_summary['donut_blur_atm_fwhm_50'][selection],
             yerr=np.array([day_obs_summary['donut_blur_atm_fwhm_50'][selection] - day_obs_summary['donut_blur_atm_fwhm_low'][selection], 
                            day_obs_summary['donut_blur_atm_fwhm_high'][selection] - day_obs_summary['donut_blur_atm_fwhm_50'][selection]]), 
             fmt='_', c='0.5', label='sqrt(donut_blur^2 - cam_fwhm^2): 10th, 50th, 90th Percentile')

plt.legend()
plt.xticks(xticks, day_obs_summary['day_obs'][selection], rotation=90)
plt.ylim(0, plt.ylim()[-1])
#for y in [0.04, 0.2, 0.4, 0.6]:
#    plt.axhline(y, c='0.75', ls='--', zorder=0)

plt.axhline(1., c='0.25', ls=':', zorder=0)
plt.axhline(0.45, c='red', ls=':', zorder=0)
plt.axhline(0.04, c='orange', ls=':', zorder=0)

plt.ylabel('Image Quality Contribution (arcsec)')
plt.tight_layout()
#plt.savefig('system_diq_timeline.pdf')

In [ ]:
from astropy.time import Time

def dayObsToTime(day_obs):
    year = str(day_obs)[0:4]
    month = str(day_obs)[4:6]
    day = str(day_obs)[6:8]
    time = Time('%s-%s-%s 12:00:00'%(year, month, day), format='iso')
    return time

selection = np.tile(True, len(day_obs_summary))

xticks = np.arange(len(day_obs_summary[selection]))
xticks = np.array([dayObsToTime(_).mjd for _ in day_obs_summary['day_obs'][selection]])

print(xticks)

fig, ax = plt.subplots(figsize=(10, 6))
a = np.linspace(0., 1., 10)

variation = ax.errorbar(
    xticks - 0.15, day_obs_summary['psf_fwhm_95_05_50'][selection],
    yerr=np.array([
        day_obs_summary['psf_fwhm_95_05_50'][selection] - day_obs_summary['psf_fwhm_95_05_low'][selection], 
        day_obs_summary['psf_fwhm_95_05_high'][selection] - day_obs_summary['psf_fwhm_95_05_50'][selection]
    ]), 
    fmt='_', c='tab:blue', label='Variation Across FoV'
    #label='sqrt(fwhm_95^2 - fwhm_5^2)'
)
aos = ax.errorbar(
    xticks + 0., day_obs_summary['aos_cam_fwhm_50'][selection],
    yerr=np.array([day_obs_summary['aos_cam_fwhm_50'][selection] - day_obs_summary['aos_cam_fwhm_low'][selection], 
                   day_obs_summary['aos_cam_fwhm_high'][selection] - day_obs_summary['aos_cam_fwhm_50'][selection]]), 
    fmt='_', c='tab:purple', label='Optics + Camera Diffusion'
    #label='sqrt(aos_fwhm^2 + cam_fwhm^2)'
)
sys = ax.errorbar(
    xticks + 0.15, day_obs_summary['sys_50_50'][selection],
    yerr=np.array([day_obs_summary['sys_50_50'][selection] - day_obs_summary['sys_50_low'][selection], 
                   day_obs_summary['sys_50_high'][selection] - day_obs_summary['sys_50_50'][selection]]), 
    fmt='_', c='tab:red', label='Delivered - Estimated Atmosphere'
    #label='sqrt(fwhm_50^2 - (donut_blur^2 - cam_fwhm^2))'
)
ellipticity = ax.errorbar(
    xticks + 0., day_obs_summary['psf_e_50_50'][selection],
    yerr=np.array([day_obs_summary['psf_e_50_50'][selection] - day_obs_summary['psf_e_50_low'][selection], 
                   day_obs_summary['psf_e_50_high'][selection] - day_obs_summary['psf_e_50_50'][selection]]), 
    fmt='_', c='tab:orange', label='PSF Ellipticity'
    #label='psf_e'
)


psf = ax.errorbar(
    xticks - 0.075, day_obs_summary['psf_fwhm_50_50'][selection],
    yerr=np.array([day_obs_summary['psf_fwhm_50_50'][selection] - day_obs_summary['psf_fwhm_50_low'][selection], 
                   day_obs_summary['psf_fwhm_50_high'][selection] - day_obs_summary['psf_fwhm_50_50'][selection]]), 
    fmt='_', c='black', label='Delivered Median PSF FWHM' 
    #label='fwhm_50'
)
atm = ax.errorbar(
    xticks + 0.075, day_obs_summary['donut_blur_atm_fwhm_50'][selection],
    yerr=np.array([day_obs_summary['donut_blur_atm_fwhm_50'][selection] - day_obs_summary['donut_blur_atm_fwhm_low'][selection], 
                   day_obs_summary['donut_blur_atm_fwhm_high'][selection] - day_obs_summary['donut_blur_atm_fwhm_50'][selection]]), 
    fmt='_', c='0.5', label='Estimated Atmosphere'
    #label='sqrt(donut_blur^2 - cam_fwhm^2)'
)

first_legend = ax.legend(handles=[variation, aos, sys], loc='upper left', title='Estimated System Contribution')
second_legend = ax.legend(handles=[psf, atm, ellipticity], loc='upper right', title='Measured Quantities')
ax.add_artist(first_legend)

ax.set_xticks(xticks, day_obs_summary['day_obs'][selection], rotation=90)
#ax.set_ylim(0, plt.ylim()[-1])
ax.set_ylim(0, 2.5)
#for y in [0.04, 0.2, 0.4, 0.6]:
#    plt.axhline(y, c='0.75', ls='--', zorder=0)

ax.axhline(1., c='0.25', ls=':', zorder=0)
ax.axhline(0.45, c='red', ls=':', zorder=0)
ax.axhline(0.04, c='orange', ls=':', zorder=0)

ax.set_ylabel('Image Quality Metric (arcsec)')
plt.title('10th, 50th, 90th Percentiles of Per-visit Quantities for Each Night')
fig.tight_layout()
#plt.savefig('system_diq_timeline.pdf')

In [ ]:
xticks = np.arange(len(day_obs_summary))

plt.figure(figsize=(10, 6))
plt.errorbar(xticks, day_obs_summary['aos_fwhm_50'],
             yerr=np.array([day_obs_summary['aos_fwhm_50'] - day_obs_summary['aos_fwhm_05'], 
                            day_obs_summary['aos_fwhm_95'] - day_obs_summary['aos_fwhm_50']]), 
             fmt='_')
plt.xticks(xticks, day_obs_summary['day_obs'], rotation=90)
plt.ylabel('aos_fwhm')
plt.tight_layout()

In [ ]:
plt.figure()
for band in bands:
    selection = (visits_summary["band"] == band)
    plt.scatter(visits_summary['donut_blur_fwhm'][selection], visits_summary['psf_fwhm_50'][selection], s=2, c=colors[band], label=band)
plt.xlim(0.5, 2.)
plt.ylim(0.5, 2.)
plt.legend(markerscale=3)
plt.xlabel('donut_blur_fwhm (arcsec)')
plt.ylabel('median_psf_fwhm (arcsec)')

In [ ]:
bins=np.linspace(-0.5, 0.5, 50)

plt.figure()
#for band in bands:
for band in ['g', 'r', 'i', 'z']:
    selection = (visits_summary["band"] == band) & (visits_summary['psf_fwhm_50'] < 2.0)
    plt.hist(visits_summary['donut_blur_fwhm'][selection] - visits_summary['psf_fwhm_50'][selection], bins=bins, density=True, histtype='step', color=colors[band], label=band, lw=2)
plt.legend()
plt.xlabel('median_psf_fwhm - donut_blur_fwhm (arcsec)')
plt.ylabel('PDF')